## Data Loading and Initialization

### Import Libraries

In [1]:
# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

This project utilises Pandas for data manipulation and analysis, and employs Matplotlib, Seaborn, and Squarify for data visualisation.

In [ ]:
!pip install pandas matplotlib seaborn squarify

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import squarify

### Load Datasets

The project uses three datasets: two sourced from Kaggle that provide data on movies and TV shows, and a third dataset on subscription plans that I collected independently.

In [4]:
movies_data = pd.read_csv('Data/movies_data.csv')
movies_data.head()

,Unnamed: 0,ID,Title,Year,Age,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type
0,0,1,The Irishman,2019,18+,98/100,1,0,0,0,0
1,1,2,Dangal,2016,7+,97/100,1,0,0,0,0
2,2,3,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0,0,0,0
3,3,4,Lagaan: Once Upon a Time in India,2001,7+,94/100,1,0,0,0,0
4,4,5,Roma,2018,18+,94/100,1,0,0,0,0


In [5]:
tv_shows = pd.read_csv('Data/tv_shows.csv')
tv_shows.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type
0,0,1,Breaking Bad,2008,18+,9.4/10,100/100,1,0,0,0,1
1,1,2,Stranger Things,2016,16+,8.7/10,96/100,1,0,0,0,1
2,2,3,Attack on Titan,2013,18+,9.0/10,95/100,1,1,0,0,1
3,3,4,Better Call Saul,2015,18+,8.8/10,94/100,1,0,0,0,1
4,4,5,Dark,2017,16+,8.8/10,93/100,1,0,0,0,1


In [6]:
svod_ad_plans = pd.read_csv('Data/svod_ad_plans.csv')
svod_ad_plans

,SN,Platform,Plan Name,Ad-Supported,Resolution,Devices Supported,Download Support,Price (US$)
0,1,Disney+,Basic,Yes,1080p,2,No,$7.99
1,2,Hulu,Hulu (With Ads),Yes,1080p,2,No,$7.99
2,3,Netflix,Standard with ads,Yes,1080p,2,Yes,$6.99
3,4,Prime Video,Prime Video,Yes,1080p,2,Yes,$8.99


**Note:** The data collected for subscription plans include only the most basic and cost-effective options from each platform, deliberately omitting bundled packages, special offers, and additional perks to ensure a uniform and objective comparison.

## Data Cleaning and Standardization

### Create New Column: Platform

For the purpose of simplifying the analysis of show availability, I'm creating a new 'Platform' column by combining the existing platform columns and filtering for shows that are available on each platform.

In [7]:
movies_streaming = (
    movies_data[['Title', 'Netflix', 'Hulu', 'Disney+', 'Prime Video']]
    # Transform the DataFrame from wide to long format
    .melt(id_vars=['Title'], var_name='Platform', value_name='Available')
    # Filter rows where 'Available' is 1 (i.e., the show is available on the platform)
    .query("Available == 1")
    # Drop the 'Available' column as it's no longer needed
    .drop(columns=['Available'])
)
movies_streaming.head()

,Title,Platform
0,The Irishman,Netflix
1,Dangal,Netflix
2,David Attenborough: A Life on Our Planet,Netflix
3,Lagaan: Once Upon a Time in India,Netflix
4,Roma,Netflix


In [8]:
# Merge the movies_streaming DataFrame with the movies_data DataFrame on 'Title' to combine streaming availability with other details
movies_expanded = movies_streaming.merge(movies_data, on='Title', how='inner')
# Drop the streaming platform columns as they are no longer needed after the merge
movies_expanded.drop(columns=['Netflix', 'Hulu', 'Prime Video', 'Disney+'], inplace=True)
movies_expanded.head()

,Title,Platform,Unnamed: 0,ID,Year,Age,Rotten Tomatoes,Type
0,The Irishman,Netflix,0,1,2019,18+,98/100,0
1,Dangal,Netflix,1,2,2016,7+,97/100,0
2,David Attenborough: A Life on Our Planet,Netflix,2,3,2020,7+,95/100,0
3,Lagaan: Once Upon a Time in India,Netflix,3,4,2001,7+,94/100,0
4,Roma,Netflix,4,5,2018,18+,94/100,0


In [9]:
tv_shows_streaming = (
    tv_shows[['Title', 'Netflix', 'Hulu', 'Disney+', 'Prime Video']]
    # Transform the DataFrame from wide to long format
    .melt(id_vars=['Title'], var_name='Platform', value_name='Available')
    # Filter rows where 'Available' is 1 (i.e., the show is available on the platform)
    .query("Available == 1")
    # Drop the 'Available' column as it's no longer needed
    .drop(columns=['Available'])
)

tv_shows_streaming.head()

,Title,Platform
0,Breaking Bad,Netflix
1,Stranger Things,Netflix
2,Attack on Titan,Netflix
3,Better Call Saul,Netflix
4,Dark,Netflix


In [10]:
# Merge the tv_shows_streaming DataFrame with the tv_shows DataFrame on 'Title' to combine streaming availability with other details
tv_shows_expanded = tv_shows_streaming.merge(tv_shows, on='Title', how='inner')
# Drop the streaming platform columns as they are no longer needed after the merge
tv_shows_expanded.drop(columns=['Netflix', 'Hulu', 'Prime Video', 'Disney+'], inplace=True)
tv_shows_expanded.head()

,Title,Platform,Unnamed: 0,ID,Year,Age,IMDb,Rotten Tomatoes,Type
0,Breaking Bad,Netflix,0,1,2008,18+,9.4/10,100/100,1
1,Stranger Things,Netflix,1,2,2016,16+,8.7/10,96/100,1
2,Attack on Titan,Netflix,2,3,2013,18+,9.0/10,95/100,1
3,Better Call Saul,Netflix,3,4,2015,18+,8.8/10,94/100,1
4,Dark,Netflix,4,5,2017,16+,8.8/10,93/100,1


### Create New Column: Maturity Rating

While the dataset provides age ratings, using standardized maturity ratings (like PG, PG-13, R) would offer a more consistent and universally recognized way to categorize content. To streamline the analysis, I'll next convert the age ratings to corresponding maturity ratings.

In [11]:
movies_expanded['Age'].unique()

array(['18+', '7+', '13+', '16+', 'all', nan], dtype=object)

In [12]:
tv_shows_expanded['Age'].unique()

array(['18+', '16+', '7+', 'all', nan, '13+'], dtype=object)

First, I'll create a function called "assign_rating" that maps age ratings to MPAA film ratings.

In [13]:
def assign_rating(age):
  if age in ['18+', '16+']:
    return 'R'
  elif age == '13+':
    return 'PG-13'
  elif age == '7+':
    return 'PG'
  elif age == 'all':
    return 'G'
  else:
    return 'N/A'

**Note:** The MPAA film rating system is based on several factors, including violence, language, sexual content, drug use, and other potentially offensive material. This function provides an approximate mapping based on age, but it's important to note that the actual rating may vary depending on the specific content.

Next, I'm going to create a new column "Maturity Rating" for both dataframes and apply the assign_rating function to assign the ratings.

In [14]:
movies_expanded['Maturity Rating'] = movies_expanded['Age'].apply(assign_rating)
movies_expanded.head()

,Title,Platform,Unnamed: 0,ID,Year,Age,Rotten Tomatoes,Type,Maturity Rating
0,The Irishman,Netflix,0,1,2019,18+,98/100,0,R
1,Dangal,Netflix,1,2,2016,7+,97/100,0,PG
2,David Attenborough: A Life on Our Planet,Netflix,2,3,2020,7+,95/100,0,PG
3,Lagaan: Once Upon a Time in India,Netflix,3,4,2001,7+,94/100,0,PG
4,Roma,Netflix,4,5,2018,18+,94/100,0,R


In [15]:
tv_shows_expanded['Maturity Rating'] = tv_shows_expanded['Age'].apply(assign_rating)
tv_shows_expanded.head()

,Title,Platform,Unnamed: 0,ID,Year,Age,IMDb,Rotten Tomatoes,Type,Maturity Rating
0,Breaking Bad,Netflix,0,1,2008,18+,9.4/10,100/100,1,R
1,Stranger Things,Netflix,1,2,2016,16+,8.7/10,96/100,1,R
2,Attack on Titan,Netflix,2,3,2013,18+,9.0/10,95/100,1,R
3,Better Call Saul,Netflix,3,4,2015,18+,8.8/10,94/100,1,R
4,Dark,Netflix,4,5,2017,16+,8.8/10,93/100,1,R


### Handle Missing Data

In [16]:
movies_expanded.isnull().sum()

Title                 0
Platform              0
Unnamed: 0            0
ID                    0
Year                  0
Age                4271
Rotten Tomatoes       7
Type                  0
Maturity Rating       0
dtype: int64

In [17]:
movies_expanded['Age'] = movies_expanded['Age'].fillna('N/A')
movies_expanded['Rotten Tomatoes'] = movies_expanded['Rotten Tomatoes'].fillna(0)

In [18]:
tv_shows_expanded.isnull().sum()

Title                 0
Platform              0
Unnamed: 0            0
ID                    0
Year                  0
Age                2192
IMDb                984
Rotten Tomatoes       0
Type                  0
Maturity Rating       0
dtype: int64

In [19]:
tv_shows_expanded['Age'] = tv_shows_expanded['Age'].fillna('N/A')
tv_shows_expanded['IMDb'] = tv_shows_expanded['IMDb'].fillna('N/A')

### Standardize Rating Data

In [20]:
# Extract the numerical part of Rotten Tomatoes ratings, which are formatted as 'x/100'
movies_expanded['Rotten Tomatoes'] = movies_expanded['Rotten Tomatoes'].str.split('/').str[0]
movies_expanded['Rotten Tomatoes'] = pd.to_numeric(movies_expanded['Rotten Tomatoes'], errors='coerce')
movies_expanded.head()

,Title,Platform,Unnamed: 0,ID,Year,Age,Rotten Tomatoes,Type,Maturity Rating
0,The Irishman,Netflix,0,1,2019,18+,98.0,0,R
1,Dangal,Netflix,1,2,2016,7+,97.0,0,PG
2,David Attenborough: A Life on Our Planet,Netflix,2,3,2020,7+,95.0,0,PG
3,Lagaan: Once Upon a Time in India,Netflix,3,4,2001,7+,94.0,0,PG
4,Roma,Netflix,4,5,2018,18+,94.0,0,R


In [21]:
# Extract the numerical part of Rotten Tomatoes ratings, which are formatted as 'x/100'
tv_shows_expanded['Rotten Tomatoes'] = tv_shows_expanded['Rotten Tomatoes'].str.split('/').str[0]
tv_shows_expanded['Rotten Tomatoes'] = pd.to_numeric(tv_shows_expanded['Rotten Tomatoes'], errors='coerce')
tv_shows_expanded.head()

,Title,Platform,Unnamed: 0,ID,Year,Age,IMDb,Rotten Tomatoes,Type,Maturity Rating
0,Breaking Bad,Netflix,0,1,2008,18+,9.4/10,100,1,R
1,Stranger Things,Netflix,1,2,2016,16+,8.7/10,96,1,R
2,Attack on Titan,Netflix,2,3,2013,18+,9.0/10,95,1,R
3,Better Call Saul,Netflix,3,4,2015,18+,8.8/10,94,1,R
4,Dark,Netflix,4,5,2017,16+,8.8/10,93,1,R


In [22]:
# Extract the numerical part of IMDb ratings, which are formatted as 'x/10'
tv_shows_expanded['IMDb'] = tv_shows_expanded['IMDb'].str.split('/').str[0]
tv_shows_expanded['IMDb'] = pd.to_numeric(tv_shows_expanded['IMDb'], errors='coerce')
tv_shows_expanded.head()

,Title,Platform,Unnamed: 0,ID,Year,Age,IMDb,Rotten Tomatoes,Type,Maturity Rating
0,Breaking Bad,Netflix,0,1,2008,18+,9.4,100,1,R
1,Stranger Things,Netflix,1,2,2016,16+,8.7,96,1,R
2,Attack on Titan,Netflix,2,3,2013,18+,9.0,95,1,R
3,Better Call Saul,Netflix,3,4,2015,18+,8.8,94,1,R
4,Dark,Netflix,4,5,2017,16+,8.8,93,1,R


### Optimize Dataset Structure

In [23]:
movies_expanded = movies_expanded[['Title', 'Platform', 'Year', 'Age', 'Maturity Rating', 'Rotten Tomatoes']]
movies_expanded.head()

,Title,Platform,Year,Age,Maturity Rating,Rotten Tomatoes
0,The Irishman,Netflix,2019,18+,R,98.0
1,Dangal,Netflix,2016,7+,PG,97.0
2,David Attenborough: A Life on Our Planet,Netflix,2020,7+,PG,95.0
3,Lagaan: Once Upon a Time in India,Netflix,2001,7+,PG,94.0
4,Roma,Netflix,2018,18+,R,94.0


In [24]:
tv_shows_expanded = tv_shows_expanded[['Title', 'Platform', 'Year', 'Age', 'Maturity Rating', 'IMDb', 'Rotten Tomatoes']]
tv_shows_expanded.head()

,Title,Platform,Year,Age,Maturity Rating,IMDb,Rotten Tomatoes
0,Breaking Bad,Netflix,2008,18+,R,9.4,100
1,Stranger Things,Netflix,2016,16+,R,8.7,96
2,Attack on Titan,Netflix,2013,18+,R,9.0,95
3,Better Call Saul,Netflix,2015,18+,R,8.8,94
4,Dark,Netflix,2017,16+,R,8.8,93
